In [1]:
from MZPackage.DataPreprocessor import DataPreprocessor
from MZPackage.Utilities import *

In [2]:
dp = DataPreprocessor()

In [ ]:
# get benign data and save 
benignData = dp.getBenignData(["Iprobe1_a","Iprobe1_b","Iprobe1_c","Iprobe2_a","Iprobe2_b","Iprobe2_c","PhaseAngle_a","PhaseAngle_b","PhaseAngle_c"])
normalizedBenignData = dp.normalizeDataSetUsingSavedParams(benignData.copy())
saveData(normalizedBenignData, "normalized_benign_data_set")

In [ ]:
# generate FDI attack data and save
FDIAttackData = dp.getRandomFDIAttackData([0,1,2],1000)
saveData(FDIAttackData, "normalized_fdi_attack_data_set")

In [4]:
# generate angle shift attack data and save
angleShiftAttackData = dp.getShiftedAngleData(["Iprobe1_a","Iprobe1_b","Iprobe1_c","Iprobe2_a","Iprobe2_b","Iprobe2_c","PhaseAngle_a","PhaseAngle_b","PhaseAngle_c"]
                                  ,["PhaseAngle_a","PhaseAngle_b","PhaseAngle_c"]
                                  ,1000, shiftRange = [1,5])
normalizedAngleShiftAttackData = dp.normalizeDataSetUsingSavedParams(angleShiftAttackData.copy())
saveData(normalizedAngleShiftAttackData, "normalized_angle_shift_attack_data_set")

100%|████████████████████████████████████████████████████████████████████████████████| 216/216 [02:17<00:00,  1.57it/s]


In [6]:
# Time sync attack
timeSyncAttackData = dp.getShiftedAngleData(["Iprobe1_a","Iprobe1_b","Iprobe1_c","Iprobe2_a","Iprobe2_b","Iprobe2_c","PhaseAngle_a","PhaseAngle_b","PhaseAngle_c"]
                                  ,["Iprobe1_a","Iprobe1_b","Iprobe1_c","PhaseAngle_a","PhaseAngle_b","PhaseAngle_c"]
                                  ,1000, shiftRange = [5,15])
normalizedTimeSyncAttackData = dp.normalizeDataSetUsingSavedParams(timeSyncAttackData.copy())
saveData(normalizedTimeSyncAttackData, "normalized_time_sync_attack_data_set")

100%|████████████████████████████████████████████████████████████████████████████████| 216/216 [02:12<00:00,  1.63it/s]


In [1]:
#tmp
from MZPackage.DataPreprocessor import DataPreprocessor
from MZPackage.Utilities import *
dp = DataPreprocessor()

# Replay attack
replayAttackData = dp.getReplayAttackData(1000)
saveData(replayAttackData, "normalized_replay_attack_data_set")